In [1]:
import os
import sys
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

from keras.layers import Flatten, Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.losses import categorical_crossentropy

import tensorflow as tf

print(f'Python: {sys.version}')

Python: 3.9.5 (tags/v3.9.5:0a7dcbd, May  3 2021, 17:27:52) [MSC v.1928 64 bit (AMD64)]


In [2]:
RANDOM_STATE = 0

In [3]:
# Working with pre trained model 

#base_model = MobileNet(input_shape=(224, 224, 3), include_top=False)
base_model = MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)


for layer in base_model.layers:
  layer.trainable = False


x = Flatten()(base_model.output)
x = Dense(units=52 , activation='softmax')(x)

# creating our model.
model = Model(base_model.input, x)

In [4]:
model.compile(optimizer='adam', loss=categorical_crossentropy , metrics=['accuracy'])

In [12]:
sessions = []
directory_template = './data/images/images/train/$session$/'
for session in os.listdir('./data/images/images/train'):
    # Find XML filename
    directory = directory_template.replace('$session$', session)
    files = os.listdir(directory)
    
    for file in files:
        # Load JPG filename
        jpg_filename = directory + file
        
        emotion_dict = {
            'emotion': session,
            'img': jpg_filename
        }
        sessions.append(emotion_dict)


In [13]:
df_sessions = pd.DataFrame(sessions)
df_sessions.index.rename('session', inplace=True)

# df_sessions_null = df_sessions.loc[df_sessions['aus'].apply(lambda x: len(x)) == 0]
# df_sessions = df_sessions.loc[df_sessions['aus'].apply(lambda x: len(x)) != 0]
# df_sessions = df_sessions.explode('aus')
# df_sessions = df_sessions.sample(frac=1, random_state=RANDOM_STATE)

# print(f'df_sessions_null: {df_sessions_null.shape}')
# print(f'df_sessions: {df_sessions.shape}')
# print()
# df_sessions.info()
# print()
# df_sessions.head()

In [19]:
df_sessions.to_csv('emotion.csv')

In [15]:
train_datagen = ImageDataGenerator(
    zoom_range = 0.2, 
    shear_range = 0.2, 
    horizontal_flip=True, 
    rescale = 1./255
)
train_data = train_datagen.flow_from_dataframe(
    dataframe = pd.concat([df_sessions] * 5).sample(frac=1),
    directory=".",
    x_col = "img",
    y_col = "aus",
    subset = "training",
    batch_size = 32,
    seed = 42,
    class_mode = "categorical",
    target_size = (128,128)
)

KeyError: 'aus'

In [ ]:
# to visualize the images in the traing data denerator 

t_img , label = train_data.next()

#-----------------------------------------------------------------------------
# function when called will prot the images 
def plotImages(img_arr, label):
  """
  input  :- images array 
  output :- plots the images 
  """
  count = 0
  for im, l in zip(img_arr,label) :
    plt.imshow(im)
    plt.title(im.shape)
    plt.axis = False
    plt.show()
    break

#-----------------------------------------------------------------------------
# function call to plot the images 
plotImages(t_img, label)

In [ ]:
ist = model.fit(
    train_data,
    steps_per_epoch=10,
    epochs=30,
    #validation_data=test_data,
    validation_steps=8,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            filepath='./results/model.h5',
            save_best_only=True,
            monitor="accuracy"
        )
    ]
)

print()
max_acc = max(hist.history['accuracy']) * 100
print(f'Accuracy: {max_acc = :.2f}%')